In [ ]:
import math
import random
import numpy as np

In [126]:
def american_to_percentage(odds):
    if odds >= 100:
        return 100 / (odds + 100)
    elif odds <= -100:
        return abs(odds) / (abs(odds) + 100)
    else:
        print("Invalid Number, cannot be between -100 and 100")
        return None
    
def percentage_to_american(probability):
    if probability == 0:
        return None
    elif probability > 0.5:
        return round((probability * 100) / (1 - probability) * -1)
    else:
        return round((100 / probability) - 100)
    
def calculate_winnings(odds, bet_amount):
    if odds > 0:
        winnings = (odds / 100) * bet_amount
    else:
        winnings = (100 / abs(odds)) * bet_amount
    
    return round(winnings, 2)

def parlay(odds_list):
    parlay_probability = 1
    for odds in odds_list:
        parlay_probability *= american_to_percentage(odds)
    return parlay_probability   
    

In [131]:
starting_amount = 100
bet_amount = 1
bets = [-110] * 20
results = []

for _ in range(10_000):
    winnings = 0
    for bet in bets:
        num = random.random()
        if num > american_to_percentage(bet):
            winnings += calculate_winnings(bet, bet_amount)
        else:
            winnings -= bet_amount
            
            
    results.append((starting_amount + winnings))
    

print(results)



[89.55, 104.83, 99.1, 95.28, 97.19, 101.01, 95.28, 91.46000000000001, 101.01, 97.19, 91.46000000000001, 99.1, 102.92, 95.28, 99.1, 104.83, 97.19, 97.19, 93.37, 99.1, 97.19, 97.19, 97.19, 95.28, 99.1, 97.19, 104.83, 106.74, 99.1, 95.28, 104.83, 101.01, 101.01, 99.1, 101.01, 97.19, 97.19, 97.19, 97.19, 101.01, 97.19, 93.37, 93.37, 101.01, 99.1, 95.28, 99.1, 104.83, 108.65, 91.46000000000001, 91.46000000000001, 97.19, 102.92, 99.1, 89.55, 108.65, 99.1, 95.28, 102.92, 101.01, 99.1, 97.19, 99.1, 99.1, 106.74, 93.37, 97.19, 97.19, 97.19, 95.28, 102.92, 99.1, 95.28, 93.37, 97.19, 99.1, 97.19, 95.28, 95.28, 106.74, 93.37, 106.74, 101.01, 87.64, 99.1, 106.74, 93.37, 104.83, 106.74, 99.1, 102.92, 93.37, 99.1, 108.65, 101.01, 99.1, 104.83, 93.37, 102.92, 101.01, 99.1, 102.92, 91.46000000000001, 97.19, 97.19, 95.28, 99.1, 97.19, 99.1, 99.1, 99.1, 102.92, 99.1, 91.46000000000001, 93.37, 102.92, 104.83, 99.1, 97.19, 97.19, 97.19, 110.56, 97.19, 93.37, 99.1, 101.01, 101.01, 97.19, 95.28, 95.28, 99.1,

In [128]:
print(calculate_winnings(-130, 1))

0.77
